In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

# Set your dataset path here
DATASET_PATH = r'C:\Users\spars\Desktop\Cloth_Defect_Detection\Cloth_Dataset'

# Set the image size to 64x64 and batch size
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32

# ImageDataGenerator for both augmented and non-augmented images
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split data into 80% training and 20% validation
)

# Load training data (all classes)
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation data (all classes)
validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Get the class indices to print them
class_names = train_generator.class_indices
print("Class Indices (Labels):", class_names)

# Calculate class weights based on the number of images per class
class_labels = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)

# Define a simple CNN model
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
model = build_model()

# Train the model with class weights to handle the imbalance
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    class_weight=class_weights_dict  # Use class weights to balance the classes
)


Found 20481 images belonging to 5 classes.
Found 5119 images belonging to 5 classes.
Class Indices (Labels): {'good': 0, 'hole': 1, 'objects': 2, 'oil spot': 3, 'thread error': 4}
Class Weights: {0: 0.22098618903754855, 1: 15.171111111111111, 2: 6.1137313432835825, 3: 8.0475442043222, 4: 8.258467741935483}


C:\Users\spars\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\spars\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


641/641 ━━━━━━━━━━━━━━━━━━━━ 294s 444ms/step - accuracy: 0.2408 - loss: 1.6632 - val_accuracy: 0.0191 - val_loss: 1.6095
Epoch 2/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 56s 86ms/step - accuracy: 0.1268 - loss: 1.5803 - val_accuracy: 0.0266 - val_loss: 1.6087
Epoch 3/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 82s 127ms/step - accuracy: 0.0895 - loss: 1.6225 - val_accuracy: 0.0205 - val_loss: 1.5875
Epoch 4/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 171s 265ms/step - accuracy: 0.0155 - loss: 1.6289 - val_accuracy: 0.0207 - val_loss: 1.5891
Epoch 5/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 56s 87ms/step - accuracy: 0.0165 - loss: 1.6443 - val_accuracy: 0.0209 - val_loss: 1.5825
Epoch 6/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 57s 89ms/step - accuracy: 0.1481 - loss: 1.6008 - val_accuracy: 0.9131 - val_loss: 1.5608
Epoch 7/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 56s 87ms/step - accuracy: 0.5114 - loss: 1.6095 - val_accuracy: 0.9121 - val_loss: 1.5772
Epoch 8/10
641/641 ━━━━━━━━━━━━━━━━━━━━ 55s 86ms/step - accuracy: 0.3174 - loss: 1.6219 - val_ac

In [11]:
model.save('cloth_detection.keras')

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight

# Set your dataset path here
DATASET_PATH = r'C:\Users\spars\Desktop\Cloth_Defect_Detection\Cloth_Dataset'

# Set the image size to 64x64 (as per your dataset)
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32

# Create ImageDataGenerator with heavy augmentation to artificially increase defective samples
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation
)

# Training generator (for defect classes only)
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=['hole', 'objects', 'oil spot', 'thread error']  # Only defect classes
)

# Validation generator (for defect classes only)
validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    classes=['hole', 'objects', 'oil spot', 'thread error']  # Only defect classes
)

# Get the class indices to print them
class_names = train_generator.class_indices
print("Class Indices (Labels):", class_names)

# Calculate class weights based on the number of images per class
class_labels = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)

# Define a CNN model for defect classification
def build_defect_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Dropout to reduce overfitting
        tf.keras.layers.Dense(4, activation='softmax')  # 4 classes (defects)
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
model = build_defect_model()

# Train the model with class weights
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # You can increase epochs later
    class_weight=class_weights_dict  # Use class weights to balance the classes
)

# Save the trained defect classification model
model.save('defect_classifier.h5')


Found 1945 images belonging to 4 classes.
Found 485 images belonging to 4 classes.
Class Indices (Labels): {'hole': 0, 'objects': 1, 'oil spot': 2, 'thread error': 3}
Class Weights: {0: 1.8009259259259258, 1: 0.7257462686567164, 2: 0.9553045186640472, 3: 0.9803427419354839}


C:\Users\spars\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\spars\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 22s 302ms/step - accuracy: 0.1963 - loss: 1.4430 - val_accuracy: 0.2948 - val_loss: 1.3852
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.2611 - loss: 1.3938 - val_accuracy: 0.3299 - val_loss: 1.3837
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.2939 - loss: 1.3812 - val_accuracy: 0.2557 - val_loss: 1.3864
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.2491 - loss: 1.3882 - val_accuracy: 0.2557 - val_loss: 1.3867
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.2584 - loss: 1.3831 - val_accuracy: 0.2557 - val_loss: 1.3863
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.2400 - loss: 1.3787 - val_accuracy: 0.2557 - val_loss: 1.3868
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.2326 - loss: 1.3796 - val_accuracy: 0.2557 - val_loss: 1.3865
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.2247 - loss: 1.3823 - val_accuracy: 0.2557 - val_loss: 